# SageMaker Model Monitoring


## Data Preparation

In [24]:
# Step 1: Setup
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split 

sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name

# S3 bucket for storing data
bucket = 'sagemaker-ml-28573'
prefix = 'model-monitor'
output_path = f's3://{bucket}/{prefix}/output'

# Load the dataset
file_path = 'Employee.csv'  # Replace with your actual file path in S3 if needed
employee_df = pd.read_csv(file_path)
employee_df.head()



# Step 2: Data Preparation
# Convert categorical columns to numeric
employee_df['Education'] = employee_df['Education'].astype('category').cat.codes
employee_df['City'] = employee_df['City'].astype('category').cat.codes
employee_df['Gender'] = employee_df['Gender'].astype('category').cat.codes
employee_df['EverBenched'] = employee_df['EverBenched'].map({'Yes': 1, 'No': 0})

# Drop rows with NaN values in the target column
employee_df.dropna(subset=['LeaveOrNot'])

# Convert target column to numeric if needed
employee_df['LeaveOrNot'] = employee_df['LeaveOrNot'].astype(int)

# Ensure no missing values in feature columns
employee_df = employee_df.dropna()

# Verify all columns are numeric
print(employee_df.dtypes)

# Define features and target
feature_columns = [
    'Education', 'JoiningYear', 'City', 'PaymentTier', 'Age',
    'Gender', 'EverBenched', 'ExperienceInCurrentDomain'
]
target_column = 'LeaveOrNot'

employee_df = employee_df[[target_column] + feature_columns]

train_df, test_df = train_test_split(employee_df, test_size=0.2, random_state=42)

# Display the transformed dataset
employee_df.head()



# Initialize S3 client
s3 = boto3.client('s3')


# Save the data locally first
train_file = 'train.csv'
validation_file = 'validation.csv'
train_df.to_csv(train_file, index=False)
test_df.to_csv(validation_file, index=False)

# Upload the data to S3
s3.upload_file(train_file, bucket, f'{prefix}/train/{train_file}')
s3.upload_file(validation_file, bucket, f'{prefix}/validation/{validation_file}')

print(f"Training data uploaded to s3://{bucket}/{prefix}/train/{train_file}")
print(f"Validation data uploaded to s3://{bucket}/{prefix}/validation/{validation_file}")




Education                     int8
JoiningYear                  int64
City                          int8
PaymentTier                  int64
Age                          int64
Gender                        int8
EverBenched                  int64
ExperienceInCurrentDomain    int64
LeaveOrNot                   int64
dtype: object
Training data uploaded to s3://sagemaker-ml-28573/model-monitor/train/train.csv
Validation data uploaded to s3://sagemaker-ml-28573/model-monitor/validation/validation.csv


In [26]:
### CAN BE SKIPPED ####

# Setup XGBoost Estimator
xgboost_container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.3-1")
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"40",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round":"50"
}

output_path = f's3://{bucket}/{prefix}/output'

estimator = Estimator(
    image_uri=xgboost_container, 
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1, 
    instance_type='ml.m5.xlarge', 
    volume_size=5,  # 5 GB 
    output_path=output_path
)

# Define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput(f"s3://{bucket}/{prefix}/train/{train_file}", content_type=content_type)
validation_input = TrainingInput(f"s3://{bucket}/{prefix}/validation/{validation_file}", content_type=content_type)

# Execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-09-03-12-15-08-548


Training data uploaded to s3://sagemaker-ml-28573/deployment/train/train.csv
Validation data uploaded to s3://sagemaker-ml-28573/deployment/validation/validation.csv
2024-09-03 12:15:10 Starting - Starting the training job...
2024-09-03 12:15:23 Starting - Preparing the instances for training...
2024-09-03 12:16:11 Downloading - Downloading the training image......
2024-09-03 12:17:02 Training - Training image download completed. Training in progress.
2024-09-03 12:17:02 Uploading - Uploading generated training model[2024-09-03 12:16:57.415 ip-10-2-209-70.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-03 12:16:57.437 ip-10-2-209-70.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-09-03:12:16:57:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-03:12:16:57:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-09-03:12:16:57:INFO] No GPUs detect

## Deploy as Real-Time Endpoint

In [130]:
from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

output_path = f's3://{bucket}/{prefix}/output'
s3_capture_upload_path = output_path
# Specify the S3 path to the pre-trained model artifact
model_artifact = "s3://sagemaker-ml-28573/deployment/output/sagemaker-xgboost-2024-09-03-12-15-08-548/output/model.tar.gz"

# Specify the endpoint name
endpoint_name = 'employee-monitor-endpoint'

# Retrieve the container image for the framework (e.g., XGBoost)
container = sagemaker.image_uris.retrieve(framework="xgboost", region=boto3.Session().region_name, version="1.3-1")

# Create the model object using the S3 path
model = Model(
    image_uri=container,
    model_data=model_artifact,
    role=role
)


data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)



INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-09-04-13-27-32-546
INFO:sagemaker:Creating endpoint-config with name employee-monitor-endpoint
INFO:sagemaker:Creating endpoint with name employee-monitor-endpoint


------!

In [230]:
import boto3

# Initialize the SageMaker runtime client
runtime_client = boto3.client('sagemaker-runtime')



# Prepare your input data (same format as before)
test_data = test_df[feature_columns].head(25)  # Select the first row of test data for prediction
test_data_csv = test_data.to_csv(index=False, header=False).strip()  # Convert to CSV format

# Invoke the endpoint directly using the runtime client
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="text/csv",  # Specify the content type
    Body=test_data_csv  # The input data as a CSV string
)

# Parse the response
result = response['Body'].read().decode('utf-8')
print(result)


0.20303820073604584
0.20303820073604584
0.3774380087852478
0.5340070128440857
0.1691509336233139
0.37122291326522827
0.6700358986854553
0.8091390132904053
0.1691509336233139
0.8091390132904053
0.1691509336233139
0.20303820073604584
0.9251188635826111
0.1691509336233139
0.35528063774108887
0.8091390132904053
0.1633274257183075
0.38120347261428833
0.36576730012893677
0.1691509336233139
0.1691509336233139
0.40060654282569885
0.7384953498840332
0.20303820073604584
0.3876122534275055



## Model Monitor: Setting Up a Baseline Using Training Data

In [132]:
# set up paths
prefix = 'model-monitor'
baseline_prefix = f"s3://{bucket}/{prefix}/baselining" 
baseline_data = f"s3://{bucket}/{prefix}/train/{train_file}" 
baseline_results = f"{baseline_prefix}/results"   


print("Baseline data uri: {}".format(baseline_data))
print("Baseline results uri: {}".format(baseline_results))

Baseline data uri: s3://sagemaker-ml-28573/model-monitor/train/train.csv
Baseline results uri: s3://sagemaker-ml-28573/model-monitor/baselining/results


In [133]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat


my_default_monitor_1 = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.t3.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor_1.suggest_baseline(
    baseline_dataset=baseline_data,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results,
    wait=True
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2024-09-04-13-31-05-215


.................2024-09-04 13:33:51.577716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-09-04 13:33:51.577755: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-09-04 13:33:53.639934: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-09-04 13:33:53.639972: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-09-04 13:33:53.640002: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-99-143.ec2.internal): /proc/driver/nvidia/version does not exist
2024-09-04 13:33:53.640334: I te

### Explore generated constraints & statistics

In [231]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Files:")
print("\n ".join(report_files))


Found Files:
s3://sagemaker-ml-28573/model monitor/baselining/results/constraints.json
 s3://sagemaker-ml-28573/model monitor/baselining/results/statistics.json


In [137]:
import pandas as pd

# Get the latest baselining job
baseline_job = my_default_monitor.latest_baselining_job

# Normalize the JSON data
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])

# Display the first 10 rows of the DataFrame
schema_df.head(10)


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.approximate_num_distinct_values,numerical_statistics.completeness,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,LeaveOrNot,Integral,3722,0,0.343632,1279.0,0.474920,0.0,1.0,2,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0,..."
1,Education,Integral,3722,0,0.265986,990.0,0.521096,0.0,2.0,3,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.2, 'cou...",0.64,2048.0,"[[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
2,JoiningYear,Integral,3722,0,2015.070661,7500093.0,1.865897,2012.0,2018.0,7,1.0,"[{'lower_bound': 2012.0, 'upper_bound': 2012.6...",0.64,2048.0,"[[2013.0, 2013.0, 2012.0, 2015.0, 2016.0, 2018..."
3,City,Integral,3722,0,0.788823,2936.0,0.837401,0.0,2.0,3,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.2, 'cou...",0.64,2048.0,"[[1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
4,PaymentTier,Integral,3722,0,2.694519,10029.0,0.566309,1.0,3.0,3,1.0,"[{'lower_bound': 1.0, 'upper_bound': 1.2, 'cou...",0.64,2048.0,"[[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0,..."
5,Age,Integral,3722,0,29.430682,109541.0,4.844176,22.0,41.0,20,1.0,"[{'lower_bound': 22.0, 'upper_bound': 23.9, 'c...",0.64,2048.0,"[[23.0, 25.0, 26.0, 39.0, 30.0, 24.0, 28.0, 26..."
6,Gender,Integral,3722,0,0.600484,2235.0,0.489799,0.0,1.0,2,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,..."
7,EverBenched,Integral,3722,0,0.100215,373.0,0.300286,0.0,1.0,2,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,ExperienceInCurrentDomain,Integral,3722,0,2.904352,10810.0,1.560180,0.0,7.0,8,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.7, 'cou...",0.64,2048.0,"[[1.0, 3.0, 4.0, 2.0, 3.0, 2.0, 1.0, 4.0, 1.0,..."


In [138]:
# Normalize the JSON data for constraints
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])

# Display the first 10 rows of the DataFrame
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,LeaveOrNot,Integral,1.0,True
1,Education,Integral,1.0,True
2,JoiningYear,Integral,1.0,True
3,City,Integral,1.0,True
4,PaymentTier,Integral,1.0,True
5,Age,Integral,1.0,True
6,Gender,Integral,1.0,True
7,EverBenched,Integral,1.0,True
8,ExperienceInCurrentDomain,Integral,1.0,True


## Analyze data with Monitoring Schedules

In [233]:
import boto3

sm_client = boto3.client('sagemaker')
endpoints = sm_client.list_endpoints()

for endpoint in endpoints['Endpoints']:
    print(endpoint['EndpointName'])


employee-monitor-endpoint
employee-predictor


In [141]:
from sagemaker.model_monitor import CronExpressionGenerator, EndpointInput, DefaultModelMonitor


# Create the monitoring schedule
schedule = my_default_monitor_1.create_monitoring_schedule(
   monitor_schedule_name=schedule_name,
   output_s3_uri=s3_report_path,
   schedule_cron_expression=CronExpressionGenerator.hourly(),
   statistics=my_default_monitor.baseline_statistics(),
   constraints=my_default_monitor.suggested_constraints(),
   enable_cloudwatch_metrics=True,
   endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint",
   )
)

# Print the schedule response
print(schedule)


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: employee-model-monitor-schedule


None


## Invoke the model with some generate sample data

In [201]:
from threading import Thread
from time import sleep, time
import boto3
import random

# Replace with your SageMaker session and the appropriate client
sm_session = boto3.Session()
runtime_client = sm_session.client("sagemaker-runtime")

# Use the name of your deployed endpoint
endpoint_name = 'employee-monitor-endpoint'


# Define a function to generate sample data
def generate_sample_data():
    # Example: [JoiningYear, City, PaymentTier, Age, Gender, EverBenched, ExperienceInCurrentDomain]
    sample = [
        random.randint(2010, 2025),  # JoiningYear
        random.randint(0, 2),        # City
        random.randint(1, 3),        # PaymentTier
        random.randint(20, 90),      # Age
        random.randint(0, 1),        # Gender
        random.randint(0, 1),        # EverBenched
        random.randint(1, 10)        # ExperienceInCurrentDomain
    ]
    return ','.join(map(str, sample))

def invoke_endpoint_with_generated_data(ep_name, runtime_client, max_invocations, max_duration):
    start_time = time()
    invocation_count = 0

    while True:
        # Check if the time limit has been reached
        if time() - start_time > max_duration:
            break

        # Generate sample data and invoke the endpoint
        payload = generate_sample_data()
        response = runtime_client.invoke_endpoint(
            EndpointName=ep_name,
            ContentType="text/csv",
            Body=payload
        )
        print(f"Payload: {payload}, Response: {response['Body'].read().decode('utf-8')}")  # Print the response
        
        invocation_count += 1

        # Check if the invocation limit has been reached
        if invocation_count >= max_invocations:
            break

        sleep(1)  # Sleep for 1 second between requests

def invoke_endpoint_limited():
    max_invocations = 15  # Stop after 3 invocations
    max_duration = 120  # Stop after 2 minutes (120 seconds)
    invoke_endpoint_with_generated_data(endpoint_name, runtime_client, max_invocations, max_duration)

# Start a thread to invoke the endpoint with generated data and given limits
thread = Thread(target=invoke_endpoint_limited)
thread.start()


Payload: 2014,0,1,30,1,1,4, Response: 0.23545874655246735

Payload: 2016,0,3,77,0,0,7, Response: 0.3164840042591095

Payload: 2022,1,3,28,0,0,2, Response: 0.3164840042591095

Payload: 2015,1,1,55,1,0,7, Response: 0.24510744214057922

Payload: 2012,0,3,68,1,0,10, Response: 0.3164840042591095

Payload: 2025,0,2,52,0,1,10, Response: 0.22797881066799164

Payload: 2020,2,1,36,0,1,7, Response: 0.23545874655246735

Payload: 2021,0,2,72,0,0,5, Response: 0.3164840042591095

Payload: 2020,1,3,69,1,0,5, Response: 0.3164840042591095

Payload: 2012,1,3,28,0,0,4, Response: 0.3164840042591095

Payload: 2010,2,3,73,1,1,8, Response: 0.22797881066799164

Payload: 2025,2,2,72,1,1,6, Response: 0.22797881066799164

Payload: 2013,0,2,34,1,1,8, Response: 0.22797881066799164

Payload: 2025,0,3,74,0,0,1, Response: 0.3164840042591095

Payload: 2022,2,3,54,0,1,8, Response: 0.22797881066799164



Should show "Scheduled"

In [234]:
desc_schedule_result = my_default_monitor_1.describe_schedule()
print("STATUS: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))

STATUS: Scheduled


In [197]:
my_default_monitor_1.list_executions()

In [235]:
import time
a_my_executions = my_default_monitor_1.list_executions()
print(
    "We've set up an hourly schedule that starts executions at the top of the hour (with a 0-20 minute buffer). We'll need to wait until the next hour for the first execution."

)

while len(mon_executions) == 0:
    my_executions = my_default_monitor_1.list_executions()
    print("Waiting for the first execution to start...")
    time.sleep(10)
else:
    my_executions = my_default_monitor_1.list_executions()
    print(f"We have {len(my_executions)} executions started.")


We've set up an hourly schedule that starts executions at the top of the hour (with a 0-20 minute buffer). We'll need to wait until the next hour for the first execution.
We have 2 executions started.


In [210]:
latest_execution = my_executions[-1]  # Latest execution's index is -1


print("Latest execution status: {}".format(latest_execution.describe()["ProcessingJobStatus"]))
print("Latest execution result: {}".format(latest_execution.describe()["ExitMessage"]))





Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 1 violations.


In [211]:
desc_schedule_result = my_default_monitor_1.describe_schedule()
print("STATUS: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))

STATUS: Scheduled


In [212]:
report_uri = latest_execution.output.destination
print("Report Uri: {}".format(report_uri))

Report Uri: s3://sagemaker-ml-28573/model-monitor/monitoring-results//employee-monitor-endpoint/employee-model-monitor-schedule/2024/09/04/14


In [178]:
print(s3_report_path)

s3://sagemaker-ml-28573/model-monitor/monitoring-results/


## View Generated Reports

In [213]:
from urllib.parse import urlparse

s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip("/")
print("Report bucket: {}".format(report_bucket))
print("Report key: {}".format(report_key))

s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: sagemaker-ml-28573
Report key: model-monitor/monitoring-results//employee-monitor-endpoint/employee-model-monitor-schedule/2024/09/04/14
Found Report Files:
model-monitor/monitoring-results//employee-monitor-endpoint/employee-model-monitor-schedule/2024/09/04/14/constraint_violations.json


### Violations report

In [228]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')

# Define your bucket name and the key
bucket_name = 'sagemaker-ml-28573'
key = 'model-monitor/monitoring-results//employee-monitor-endpoint/employee-model-monitor-schedule/2024/09/04/14/constraint_violations.json'

# Try to download the file
try:
    s3.download_file(bucket_name, key, 'constraint_violations.json')
    print("File downloaded successfully.")
    
    # Load and process the JSON file
    with open('constraint_violations.json', 'r') as f:
        violations = json.load(f)

    constraints_df = pd.json_normalize(violations['violations'])
    pd.set_option("display.max_colwidth", None)
    print(constraints_df.head(10))

except Exception as e:
    print(f"An error occurred: {e}")


File downloaded successfully.
      feature_name constraint_check_type  \
0  Missing columns  missing_column_check   

                                                                                                                           description  
0  There are missing columns in current dataset. Number of columns in current dataset: 8, Number of columns in baseline constraints: 9  


In [229]:
import json
import pandas as pd

# Load the JSON data from the file
with open('constraint_violations.json', 'r') as f:
    violations = json.load(f)
    
# Convert the 'violations' section of the JSON data into a DataFrame
constraints_df = pd.json_normalize(violations['violations'])

# Set Pandas options to display the full width of each column
pd.set_option("display.max_colwidth", None)

# Display the first few rows of the DataFrame
constraints_df.head(10)


,feature_name,constraint_check_type,description
0,Missing columns,missing_column_check,"There are missing columns in current dataset. Number of columns in current dataset: 8, Number of columns in baseline constraints: 9"


In [236]:
my_default_monitor_1.stop_monitoring_schedule() 

INFO:sagemaker:Stopping Monitoring Schedule with name: employee-model-monitor-schedule


In [238]:
my_default_monitor_1.stop_monitoring_schedule()
my_default_monitor_1.delete_monitoring_schedule()
time.sleep(60)  # Wait for the deletion

INFO:sagemaker:Stopping Monitoring Schedule with name: employee-model-monitor-schedule
INFO:sagemaker:Deleting Monitoring Schedule with name: employee-model-monitor-schedule
INFO:sagemaker.model_monitor.model_monitoring:Deleting Data Quality Job Definition with name: data-quality-job-definition-2024-09-04-13-37-47-080
